In [2]:
import csv
import pandas as pd
import numpy as np
#from FHMM import FHMM
#from HMM import HMM, HMM_MAD, perc_std_expl,r2
#from Preprocessing import Create_combined_states, Appliance, train_test_split,create_matrix
import math


##could define R functions after getting predictionsea
def perc_std_expl_full(pred_df,obs_df):
    """
    :param observed: df of observed energy levels per channel
    :param predicted: df of predicted energy levels per channel
    :return: percentage of standard deviation explained
    """
    return_dict = {}
    for channel in pred_df:
        X_pred = pred_df[[channel]].values
        X_obs = obs_df[[channel]].values
        obs_mean = np.mean(X_obs)
        r2 = 1 - (np.sum((X_obs - X_pred)**2))/np.sum((X_obs - obs_mean)**2)
        return_dict[channel] = 1 - math.sqrt(1-r2)
    return return_dict

def r2_full(pred_df,obs_df):
    return_dict = {}
    for channel in pred_df:
        X_pred = pred_df[[channel]].values
        X_obs = obs_df[[channel]].values
        obs_mean = np.mean(X_obs)
        r2 = 1 - (np.sum((X_obs - X_pred)**2))/np.sum((X_obs - obs_mean)**2)
        return_dict[channel] = r2
    return return_dict



class Appliance():
    def __init__(self, name, power_data):
        self.name=name
        self.power_data = power_data
        self.good_chunks = power_data[(power_data[name]>1)]
        
        

def create_matrix(appliance, good_chunks = True):
    if not good_chunks:
        power_data = appliance.power_data
    else:
        power_data = appliance.good_chunks
    
    return power_data.values.reshape((-1,1))


In [3]:
## HMM.py


import pandas as pd
import numpy as np
from matplotlib import cm, pyplot as plt
from matplotlib.dates import YearLocator, MonthLocator
import cPickle as pk
from hmmlearn.hmm import GaussianHMM
import math

def HMM_MAD(model,obs_levels):
    hidden_states = model.predict(obs_levels)
    means = model.means_.round().astype(int).flatten().tolist()
    predict_levels = np.array([means[state] for state in hidden_states]).reshape(obs_levels.shape)
    abs_error = np.absolute(obs_levels - predict_levels)
    return np.mean(abs_error)/np.mean(obs_levels)

def perc_std_expl(model,obs_levels):
    """
    :param observed: df of observed energy levels per channel
    :param predicted: df of predicted energy levels per channel
    :return: percentage of standard deviation explained
    """
    hidden_states = model.predict(obs_levels)
    means = model.means_.round().astype(int).flatten().tolist()
    predict_levels = np.array([means[state] for state in hidden_states]).reshape(obs_levels.shape)
    obs_mean = np.mean(obs_levels)
    r2 = 1 - (np.sum((obs_levels - predict_levels)**2))/np.sum((obs_levels - obs_mean)**2)
    return 1 - math.sqrt(1-r2)

def r2(model,obs_levels):
    hidden_states = model.predict(obs_levels)
    means = model.means_.round().astype(int).flatten().tolist()
    predict_levels = np.array([means[state] for state in hidden_states]).reshape(obs_levels.shape)
    obs_mean = np.mean(obs_levels)
    r2 = 1 - (np.sum((obs_levels - predict_levels)**2))/np.sum((obs_levels - obs_mean)**2)
    return r2

class HMM():
    def __init__(self, X_test, X_train):
        self.X_train = X_train
        self.X_test = X_test
        self.n_states = None
        self.model =  None

    def fit_HMM(self,error_metric):
        print "Looking for optimal number of states and fitting HMM"
        for i in xrange(2,5):
            candidate = GaussianHMM(n_components=i, covariance_type="full", n_iter=1000)
            candidate.fit(self.X_train)
            if error_metric == HMM_MAD:
                error = HMM_MAD(candidate,self.X_test)
                if i == 2:
                    best_guess = error
                    best_model = candidate
                    opt_n_states = i
                else:
                    if error < best_guess:
                        opt_n_states = i
                        best_model = candidate
                        best_guess = error
            else:
                error = error_metric(candidate,self.X_test)
                if i == 2:
                    best_guess = error
                    best_model = candidate
                    opt_n_states = i
                else:
                    if error > best_guess:
                        opt_n_states = i
                        best_model = candidate
                        best_guess = error
        self.model = best_model
        self.n_states = opt_n_states
        print "Done. Lowest error of {} achieved with {} states".format(best_guess, opt_n_states)

    def extract_means(self):
        return self.model.means_[:,0].flatten()

    def HMM_total_accuracy(self, obs_levels, state_means):
        hidden_states = self.model.predict(obs_levels)
        predict_levels = [state_means[state] for state in hidden_states]
        test_error = 1 - (np.sum(obs_levels[:,0]) - np.sum(predict_levels))/np.sum(obs_levels[:,0])
        return test_error

    def HMM_MAD_perc(self,obs_levels, state_means):
        hidden_states = self.model.predict(obs_levels)
        predict_levels = np.array([state_means[state] for state in hidden_states]).reshape(obs_levels.shape)
        abs_error = np.absolute(obs_levels - predict_levels)
        return np.mean(abs_error)/np.mean(obs_levels)

    def run(self):
        self.fit_HMM()
        state_means = self.extract_means()
        test_error = self.HMM_accuracy(self.X_test, state_means)
        print "Accuracy for the model with {} hidden states is: {}".format(self.n_states,test_error)

        


In [4]:
## FHMM.py


import pandas as pd
import numpy as np
import itertools
from hmmlearn.hmm import GaussianHMM
from collections import OrderedDict
from copy import deepcopy
from Preprocessing import cluster

def compute_A_fhmm(list_A):
    result = list_A[0]
    for i in range(len(list_A) - 1):
        result = np.kron(result, list_A[i + 1])
    return result

def combine_parameters(ind_parameters_list):
    '''
    Function to compute factorized HMM parameters such as starting probabilities or transition matrices
    from individual model parameters
    :param ind_parameters_list: parameter list for individual models (per appliance), e.g. list of starting probabilites,
    transition matrices
    :return: matrix of factorized model parameters
    '''
    FactAParam = ind_parameters_list[0]
    for idx in xrange(1,len(ind_parameters_list)):
        FactA = np.kron(FactAParam,ind_parameters_list[idx])
    return FactAParam

def compute_pi_fhmm(list_pi):
    """
    Parameters
    -----------
    list_pi : List of PI's of individual learnt HMMs
    Returns
    -------
    result : Combined Pi for the FHMM
    """
    result = list_pi[0]
    for i in range(len(list_pi) - 1):
        result = np.kron(result, list_pi[i + 1])
    return result

def combine_means(means_list):
    """
    Compute factorized model states means
    :param means_list: list of individual models' state means
    :return: column vector of fatorized state means
    """
    states_combination = list(itertools.product(*means_list))
    num_combinations = len(states_combination)
    means_stacked = np.array([sum(x) for x in states_combination])
    means = np.reshape(means_stacked, (num_combinations, 1))
    cov = np.tile(5 * np.identity(1), (num_combinations, 1, 1))
    return [means, cov]

def sort_startprob(mapping, startprob):
    """ Sort the startprob according to power means; as returned by mapping
    """
    num_elements = len(startprob)
    new_startprob = np.zeros(num_elements)
    for i in xrange(len(startprob)):
        new_startprob[i] = startprob[mapping[i]]
    return new_startprob


def sort_covars(mapping, covars):
    new_covars = np.zeros_like(covars)
    for i in xrange(len(covars)):
        new_covars[i] = covars[mapping[i]]
    return new_covars


def sort_transition_matrix(mapping, A):
    """Sorts the transition matrix according to increasing order of
    power means; as returned by mapping
    Parameters
    ----------
    mapping :
    A : numpy.array of shape (k, k)
        transition matrix
    """
    num_elements = len(A)
    A_new = np.zeros((num_elements, num_elements))
    for i in range(num_elements):
        for j in range(num_elements):
            A_new[i, j] = A[mapping[i], mapping[j]]
    return A_new


def sort_learnt_parameters(startprob, means, covars, transmat):
    mapping = return_sorting_mapping(means)
    means_new = np.sort(means, axis=0)
    startprob_new = sort_startprob(mapping, startprob)
    covars_new = sort_covars(mapping, covars)
    transmat_new = sort_transition_matrix(mapping, transmat)
    assert np.shape(means_new) == np.shape(means)
    assert np.shape(startprob_new) == np.shape(startprob)
    assert np.shape(transmat_new) == np.shape(transmat)

    return [startprob_new, means_new, covars_new, transmat_new]


def return_sorting_mapping(means):
    means_copy = deepcopy(means)
    means_copy = np.sort(means_copy, axis=0)

    # Finding mapping
    mapping = {}
    for i, val in enumerate(means_copy):
        mapping[i] = np.where(val == means)[0][0]
    return mapping

def create_combined_hmm(model):
    list_pi = [model[appliance].startprob_ for appliance in model]
    list_A = [model[appliance].transmat_ for appliance in model]
    list_means = [model[appliance].means_.flatten().tolist()
                  for appliance in model]

    pi_combined = compute_pi_fhmm(list_pi)
    A_combined = compute_A_fhmm(list_A)
    [mean_combined, cov_combined] = combine_means(list_means)

    combined_model = GaussianHMM(
        n_components=len(pi_combined), covariance_type='full',
        startprob_prior=pi_combined, transmat_prior=A_combined)
    combined_model.covars_ = cov_combined
    combined_model.means_ = mean_combined
    combined_model.startprob_ = pi_combined
    combined_model.transmat_ = A_combined
    return combined_model


def decode_hmm(length_sequence, centroids, appliance_list, states):
    """
    Decodes the HMM state sequence
    """
    hmm_states = {}
    hmm_power = {}
    total_num_combinations = 1

    for appliance in appliance_list:
        total_num_combinations *= len(centroids[appliance])

    for appliance in appliance_list:
        hmm_states[appliance] = np.zeros(length_sequence, dtype=np.int)
        hmm_power[appliance] = np.zeros(length_sequence)

    for i in range(length_sequence):

        factor = total_num_combinations
        for appliance in appliance_list:
            # assuming integer division (will cause errors in Python 3x)
            factor = factor // len(centroids[appliance])

            temp = int(states[i]) / factor
            hmm_states[appliance][i] = temp % len(centroids[appliance])
            hmm_power[appliance][i] = centroids[appliance][hmm_states[appliance][i]]
    return [hmm_states, hmm_power]

class FHMM():
    """
    Attributes
    ----------
    model : dict
    predictions : pd.DataFrame()
    meters : list
    MIN_CHUNK_LENGTH : int
    """

    def __init__(self):
        self.model = {}
        self.predictions = pd.DataFrame()
        self.MIN_CHUNK_LENGTH = 100
        self.MODEL_NAME = 'FHMM'


    def train(self, appliances, num_states_dict={}, **load_kwargs):
        """Train using 1d FHMM.
        Places the learnt model in `model` attribute
        The current version performs training ONLY on the first chunk.
        Online HMMs are welcome if someone can contribute :)
        Assumes all pre-processing has been done.
        """
        learnt_model = OrderedDict()
        num_meters = len(appliances)
        if num_meters > 12:
            max_num_clusters = 2
        else:
            max_num_clusters = 3

        for i, app in enumerate(appliances):
            power_data = app.power_data.fillna(value = 0,inplace = False)
            X = power_data.values.reshape((-1, 1))
            assert X.ndim == 2
            self.X = X

            if num_states_dict.get(app.name) is not None:
                # User has specified the number of states for this appliance
                num_total_states = num_states_dict.get(app.name)

            else:
                # Find the optimum number of states
                print "Identifying number of hidden states for appliance {}".format(app.name)
                states = cluster(X, max_num_clusters)
                num_total_states = len(states)
                print "Number of hidden states for appliance {}: {}".format(app.name, num_total_states)

            print("Training model for appliance {} with {} hidden states".format(app.name, num_total_states))
            learnt_model[app.name] = GaussianHMM(num_total_states, "full")

            # Fit
            learnt_model[app.name].fit(X)


        # Combining to make a AFHMM
        self.meters = []
        new_learnt_models = OrderedDict()
        for app in learnt_model:
            startprob, means, covars, transmat = sort_learnt_parameters(
                learnt_model[app].startprob_, learnt_model[app].means_,
                learnt_model[app].covars_, learnt_model[app].transmat_)
            new_learnt_models[app] = GaussianHMM(
                startprob.size, "full")
            new_learnt_models[app].means_ = means
            new_learnt_models[app].covars_ = covars
            new_learnt_models[app].startprob_ = startprob
            new_learnt_models[app].transmat_ = transmat
            # UGLY! But works.
            self.meters.append(app)

        learnt_model_combined = create_combined_hmm(new_learnt_models)
        self.individual = new_learnt_models
        self.model = learnt_model_combined

    def disaggregate_chunk(self, test_mains):
        """Disaggregate the test data according to the model learnt previously
        Performs 1D FHMM disaggregation.
        For now assuming there is no missing data at this stage.
        :param test_mains: test dataframe with aggregate data
        """

        # Array of learnt states
        learnt_states_array = []
        test_mains = test_mains.dropna()
        length = len(test_mains.index)
        temp = test_mains.values.reshape(length, 1)
        learnt_states_array.append(self.model.predict(temp))

        # Model
        means = OrderedDict()
        for elec_meter, model in self.individual.iteritems():
            means[elec_meter] = (
                model.means_.round().astype(int).flatten().tolist())
            means[elec_meter].sort()

        decoded_power_array = []
        decoded_states_array = []

        for learnt_states in learnt_states_array:
            [decoded_states, decoded_power] = decode_hmm(
                len(learnt_states), means, means.keys(), learnt_states)
            decoded_states_array.append(decoded_states)
            decoded_power_array.append(decoded_power)

        prediction = pd.DataFrame(
            decoded_power_array[0], index=test_mains.index)

        return prediction


    def disaggregate(self, mains, output_datastore, sample_period = 20):
        '''Disaggregate mains according to the model learnt previously.
        Parameters
        ----------
        mains : dataframe with aggregated output
        output_datastore : instance of nilmtk.DataStore subclass
            For storing power predictions from disaggregation algorithm.
        sample_period : number, optional
            The desired sample period in minutes.
        **load_kwargs : key word arguments
            Passed to `mains.power_series(**kwargs)`
        '''
        for g, chunk in mains.groupby(np.arange(len(mains)) // sample_period):
            predictions = self.disaggregate_chunk(chunk)
            output_datastore = output_datastore.append(predictions)
            if g != 0 and g%72 == 0:
                print "Disaggregated {} day(s) of data".format(g/72)
        return output_datastore\

In [6]:
with open('C:/Users/yli/Desktop/energy disaggregation/Basic model/formated.iawe.data/short.df.0713-0804.csv') as f:
    combined = pd.read_csv(f)
combined.shape
type(combined)    

print combined
#combined = total['2013-06-01 00:00:00': '2013-09-30 23:59:59'][['channel_12','channel_5','channel_6','

                 time  air conditioner1  air conditioner2  washing machine  \
0      7/13/2013 0:01               0.0               0.0              0.0   
1      7/14/2013 0:01               0.0               0.0              0.0   
2      7/15/2013 0:01               0.0               0.0              0.0   
3      7/16/2013 0:01               0.0               0.0              0.0   
4      7/17/2013 0:01               0.0               0.0              0.0   
5      7/18/2013 0:01               0.0               0.0              0.0   
6      7/19/2013 0:01               0.0               0.0              0.0   
7      7/20/2013 0:01               0.0               0.0              0.0   
8      7/21/2013 0:01               0.0               0.0              0.0   
9      7/22/2013 0:01               0.0               0.0              0.0   
10     7/23/2013 0:01               0.0               0.0              0.0   
11     7/24/2013 0:01               0.0               0.0       

In [38]:
#combined.columns[1:-1]

Index([u'air conditioner1', u'air conditioner2', u'washing machine',
       u'laptop computer', u'television'],
      dtype='object')

In [23]:
with open('C:/Users/yli/Desktop/energy disaggregation/Basic model/formated.iawe.data/train.csv') as f:
    train_set = pd.read_csv(f)
    
with open('C:/Users/yli/Desktop/energy disaggregation/Basic model/formated.iawe.data/test.csv') as f:
    test_set = pd.read_csv(f)
#print train_set[['air conditioner1']]
#print test_set[['television']]
print test_set

                 time  air conditioner1  air conditioner2  washing machine  \
0      7/29/2013 0:00               0.0               0.0              0.0   
1      7/29/2013 0:01               0.0               0.0              0.0   
2      7/29/2013 0:02               0.0               0.0              0.0   
3      7/29/2013 0:03               0.0               0.0              0.0   
4      7/29/2013 0:04               0.0               0.0              0.0   
5      7/29/2013 0:05               0.0               0.0              0.0   
6      7/29/2013 0:06               0.0               0.0              0.0   
7      7/29/2013 0:07               0.0               0.0              0.0   
8      7/29/2013 0:08               0.0               0.0              0.0   
9      7/29/2013 0:09               0.0               0.0              0.0   
10     7/29/2013 0:10               0.0               0.0              0.0   
11     7/29/2013 0:11               0.0               0.0       

In [26]:
app_train_list = []
app_test_list = []

print Appliance('air conditioner1',train_set[['air conditioner1']]).good_chunks

       air conditioner1
656         1075.787875
657          395.366217
658          396.637569
659          765.930067
660         1677.935483
661         1782.302158
662         1826.711900
663         1850.200250
664         1860.634707
665         1859.967475
666         1865.927500
667         1853.444617
668         1857.175411
669         1866.131150
670         1872.481717
671         1865.324322
672         1867.373783
673         1865.090883
674         1874.780729
675         1855.380424
676         1866.622300
677         1868.850550
678         1867.524482
679         1868.010617
680         1868.792167
681         1871.168034
682         1865.011649
683         1867.568158
684         1859.206704
685         1872.053186
...                 ...
22357       1744.744200
22358       1751.196035
22359       1740.503600
22360       1730.134983
22361       1725.536068
22362       1720.706733
22363       1716.701200
22364       1719.058450
22365       1725.723018
22366       1717

In [27]:
print combined.columns

Index([u'time', u'air conditioner1', u'air conditioner2', u'washing machine',
       u'laptop computer', u'television', u'total'],
      dtype='object')


In [28]:
for app in combined.columns[1:-1]:
    app_train_list.append(Appliance(app,train_set[[app]]))
    app_test_list.append(Appliance(app,test_set[[app]]))
    
num_states_dict = {}
ModelDict = {}

print app_train_list

[<__main__.Appliance instance at 0x000000000C83BEC8>, <__main__.Appliance instance at 0x000000000C6B94C8>, <__main__.Appliance instance at 0x000000000C6B9BC8>, <__main__.Appliance instance at 0x000000000C587D88>, <__main__.Appliance instance at 0x000000000C587E48>]


In [29]:
for i, app in enumerate(app_train_list):
    X_train = create_matrix(app,good_chunks = True)
    X_test = create_matrix(app_test_list[i], good_chunks = False)
    hmm = HMM(X_train, X_test)
    print app.name
    hmm.fit_HMM(perc_std_expl)
    ModelDict[app.name] = hmm.model
    num_states_dict[app.name] = hmm.n_states

air conditioner1
Looking for optimal number of states and fitting HMM


C:\Users\yli\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\yli\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\yli\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed 

Done. Lowest error of 0.393882874677 achieved with 3 states
air conditioner2
Looking for optimal number of states and fitting HMM


C:\Users\yli\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\yli\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\yli\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed 

Done. Lowest error of 0.689481900608 achieved with 4 states
washing machine
Looking for optimal number of states and fitting HMM


...

Done. Lowest error of -0.00026724660246 achieved with 3 states
laptop computer
Looking for optimal number of states and fitting HMM


C:\Users\yli\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\yli\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\yli\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed 

Done. Lowest error of 0.307184865107 achieved with 3 states
television
Looking for optimal number of states and fitting HMM


C:\Users\yli\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\yli\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\yli\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed 

Done. Lowest error of -0.188231805586 achieved with 4 states


In [30]:
fhmm = FHMM()
fhmm.train(app_train_list,num_states_dict = num_states_dict)
predictions = pd.DataFrame()
predictions = fhmm.disaggregate(test_set[['total']],predictions)


Training model for appliance air conditioner1 with 3 hidden states


C:\Users\yli\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)


Training model for appliance air conditioner2 with 4 hidden states


C:\Users\yli\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)


Training model for appliance washing machine with 3 hidden states
Training model for appliance laptop computer with 3 hidden states


C:\Users\yli\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\yli\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)


Training model for appliance television with 4 hidden states


C:\Users\yli\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)


Disaggregated 1 day(s) of data
Disaggregated 2 day(s) of data
Disaggregated 3 day(s) of data
Disaggregated 4 day(s) of data
Disaggregated 5 day(s) of data
Disaggregated 6 day(s) of data


In [31]:
total_power_predicted = predictions.sum()
total_power_act = test_set[predictions.columns].sum()

In [32]:
print "Percent stand.dev.explained, 1min:", perc_std_expl_full(predictions, test_set)
print "R2, 1min:", r2_full(predictions, test_set)

Percent stand.dev.explained, 1min: {'air conditioner2': 0.225808701926014, 'laptop computer': 0.18158541309225207, 'television': 0.058370127659651105, 'air conditioner1': 0.42364010258867413, 'washing machine': -2.1976209834096925}
R2, 1min: {'air conditioner2': 0.40062783398651647, 'laptop computer': 0.33019756393662036, 'television': 0.11333318351629829, 'air conditioner1': 0.66780926865600587, 'washing machine': -9.2247799535419688}


In [83]:
#predictions_15Min = predictions.resample("15Min")

In [34]:
print total_power_predicted
print total_power_act

air conditioner1    1158960.0
air conditioner2    1180554.0
laptop computer      139390.0
television           185883.0
washing machine      113192.0
dtype: float64
air conditioner1    1.628854e+06
air conditioner2    8.781208e+05
laptop computer     1.269026e+05
television          1.089918e+05
washing machine     1.483415e+04
dtype: float64


In [35]:
print predictions

       air conditioner1  air conditioner2  laptop computer  television  \
0                   0.0               0.0             25.0        72.0   
1                   0.0               0.0              0.0         0.0   
2                   0.0               0.0              0.0         0.0   
3                   0.0               0.0              0.0         0.0   
4                   0.0               0.0              0.0         0.0   
5                   0.0               0.0              0.0         0.0   
6                   0.0               0.0              0.0         0.0   
7                   0.0               0.0              0.0         0.0   
8                   0.0               0.0              0.0         0.0   
9                   0.0               0.0              0.0         0.0   
10                  0.0               0.0              0.0         0.0   
11                  0.0               0.0              0.0         0.0   
12                  0.0               

In [36]:
# write out predictions dataframe

predictions.to_csv('prediction.csv')

In [ ]:
## Create graphs

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Int64Index'